In [1]:
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import xarray as xr
import pandas as pd
import scipy
from datetime import datetime, timedelta
import time
import seaborn
import matplotlib.dates as mdates
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
import math
import netCDF4 as nc
import requests

import chemO_funcs #import o2sat
from importlib import reload
import warnings
warnings.filterwarnings('ignore')

%cd phyto
%run PLT.py


/Users/akbaskind/Documents/Python Scripts/phyto


# Testing getting Hydrocat data using my `PLT` function

In [33]:
df = PLT.get_hydrocat('2022-01-01', '2022-12-30', 'PLT')
df

,TmStamp,hydrocatStart,hydrocatTemperature,hydrocatConductivity,hydrocatDissOxygen,hydrocatSalinity,hydrocatFluorescence,hydrocatTurbidity,hydrocatPH,DateTime,pH total
326,2022-01-04T09:31:15.400Z,None,6.5078,31.0263,9.592,30.7871,7.478,30.272,8.04,2022-01-04 04:31:15.400000,7.934479
331,2022-01-04T10:46:15.400Z,None,6.3631,30.9445,9.663,30.8278,1.844,3.473,8.04,2022-01-04 05:46:15.400000,7.934814
332,2022-01-04T11:01:15.200Z,None,6.3063,30.8235,9.633,30.7459,2.373,15.717,8.04,2022-01-04 06:01:15.200000,7.934773
333,2022-01-04T11:16:15.200Z,None,6.2367,30.7264,9.680,30.7015,3.158,5.010,8.04,2022-01-04 06:16:15.200000,7.934821
334,2022-01-04T11:31:15.200Z,None,6.2593,30.6975,9.698,30.6492,2.170,7.818,8.03,2022-01-04 06:31:15.200000,7.924688
...,...,...,...,...,...,...,...,...,...,...,...
31734,2022-12-29T22:46:29.000Z,12/29/2022 22:45:00,3.9276,27.7203,10.546,29.3688,0.097,64.257,8.12,2022-12-29 17:46:29,8.016546
31735,2022-12-29T23:01:29.000Z,12/29/2022 23:00:00,3.9210,27.7232,10.552,29.3782,0.093,64.640,8.11,2022-12-29 18:01:29,8.006574
31736,2022-12-29T23:16:29.000Z,12/29/2022 23:15:00,3.9183,27.7297,10.567,29.3882,0.123,64.546,8.12,2022-12-29 18:16:29,8.016597
31737,2022-12-29T23:31:29.000Z,12/29/2022 23:30:00,3.9342,27.7421,10.652,29.3883,0.123,65.017,8.13,2022-12-29 18:31:29,8.026570


# Setting up API retrieval from Google Sheets

> Make sure to have installed the necessary packages in your conda environment
>>```
conda activate base
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
pip install gspread oauth2client df2gspread

> Have `credentials.json` in your working directory

> Have `servicecredentials.json` in your working directory

> Share your Google sheet with your service account: `google-sheets@third-faculty-385421.iam.gserviceaccount.com`

> Get spreadsheet key from your Google sheet and identify the worksheet you want

## This header will always be the same

In [4]:
#The scope is always look like this so we did not need to change anything
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
#Name of our Service Account Key
google_key_file = 'servicecredentials.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

# Retrieve lab samples from Google

In [22]:
spreadsheet_key = '17FFbtUuhUS4UtxB-OjKIP2wCYJoEAmaW6VaHQPcup9U'
wks_name = 'Sheet1'

#Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key(spreadsheet_key)
#Selecting which sheet to pulling the data
sheet = workbook.worksheet(wks_name)

#Pulling the data and transform it to the data frame
values = sheet.get_all_values()
pull_tips = pd.DataFrame(values[1:], columns = values[0])
pull_tips

,Sample,Time,Location,depth,TA Temp (degC),TA (uM),DIC Temp (degC),DIC (uM),Salinity,"NOTE: Salinity values are currently either derived from buoy data or estimated to be 30 PSU. Abby will measure salinity in lab/update salinities with in situ observations once the semester is over. Additionally, some of the samples here haven't been run for TA yet. Abby will likely run that in the next couple of weeks."
0,PLT 5/9/22 surf A,5/9/22 7:30,PLT,surface,20.3,2075.47,22,1985.40452,30,
1,PLT 5/9/22 surf B,5/9/22 7:30,PLT,surface,20.3,2076.78,22,1955.71939,30,
2,PLT 4/28/22 A,4/28/22 7:30,PLT,surface,20.9,2078.825,22,1931.63783,30,
3,PLT 4/28/22 B,4/28/22 7:30,PLT,surface,20.9,2078.41,22,1931.62999,30,
4,PLT 5/22/22 bottom,5/22/22 7:30,PLT,bottom,20.9,2138.395,22,1978.53868,30,
5,GB B 9/9/22,9/9/22 11:30,GB,surface,21.3,1957.87,22.5,1883,28.6972,
6,GB A 9/9/22,9/9/22 11:30,GB,surface,21.3,1955.84,22.5,1882.8,28.6972,
7,GB B 9/20/22,9/20/22 11:30,GB,surface,21.3,2051.6,22.5,1938.6,29.4532,
8,GB A 9/20/22,9/20/22 11:30,GB,surface,21.3,2051.93,22.5,1937.2,29.4532,
9,GB A 10/25/22,10/25/22 11:30,GB,surface,21.3,2047.245,21.1,1931.6,28.738,


In [40]:
df = pull_tips[['Sample', 'Time', 'Location', 'depth', 'TA Temp (degC)', 'TA (uM)',
       'DIC Temp (degC)', 'DIC (uM)', 'Salinity']]

## For some reason everything saved as a string so convert to appropriate data type

In [63]:
df['TA (uM)'] = pd.to_numeric(df['TA (uM)'])
df['TA Temp (degC)'] = pd.to_numeric(df['TA Temp (degC)'])
df['DIC (uM)'] = pd.to_numeric(df['DIC (uM)'])
df['DIC Temp (degC)'] = pd.to_numeric(df['DIC Temp (degC)'])
df['Salinity'] = pd.to_numeric(df['Salinity'])
df["DateTime"] = pd.to_datetime(df["Time"])
df.head()

,Sample,Time,Location,depth,TA Temp (degC),TA (uM),DIC Temp (degC),DIC (uM),Salinity,DateTime
0,PLT 5/9/22 surf A,5/9/22 7:30,PLT,surface,20.3,2075.470,22.0,1985.40452,30.0,2022-05-09 07:30:00
1,PLT 5/9/22 surf B,5/9/22 7:30,PLT,surface,20.3,2076.780,22.0,1955.71939,30.0,2022-05-09 07:30:00
2,PLT 4/28/22 A,4/28/22 7:30,PLT,surface,20.9,2078.825,22.0,1931.63783,30.0,2022-04-28 07:30:00
3,PLT 4/28/22 B,4/28/22 7:30,PLT,surface,20.9,2078.410,22.0,1931.62999,30.0,2022-04-28 07:30:00
4,PLT 5/22/22 bottom,5/22/22 7:30,PLT,bottom,20.9,2138.395,22.0,1978.53868,30.0,2022-05-22 07:30:00


## Drop missing data

In [64]:
for ind in df.index:
    if math.isnan(df['TA (uM)'][ind]) or math.isnan(df['DIC (uM)'][ind]):
        df = df.drop(ind)

## Convert units
* requires `gsw`

In [67]:
df['P'] = 0

In [68]:
df['rho_DIC'] = gsw.rho(df['Salinity'], df['DIC Temp (degC)'], df['P'])
df['rho_TA'] = gsw.rho(df['Salinity'], df['TA Temp (degC)'], df['P'])

df['DIC (umol/kg)'] = df['DIC (uM)']/df['rho_DIC']/0.001
df['TA (umol/kg)'] = df['TA (uM)']/df['rho_TA']/0.001


df.head()

,Sample,Time,Location,depth,TA Temp (degC),TA (uM),DIC Temp (degC),DIC (uM),Salinity,DateTime,P,rho_DIC,rho_TA,DIC (umol/kg),TA (umol/kg)
0,PLT 5/9/22 surf A,5/9/22 7:30,PLT,surface,20.3,2075.470,22.0,1985.40452,30.0,2022-05-09 07:30:00,0,1020.366971,1020.813221,1945.774977,2033.153526
1,PLT 5/9/22 surf B,5/9/22 7:30,PLT,surface,20.3,2076.780,22.0,1955.71939,30.0,2022-05-09 07:30:00,0,1020.366971,1020.813221,1916.682375,2034.436817
2,PLT 4/28/22 A,4/28/22 7:30,PLT,surface,20.9,2078.825,22.0,1931.63783,30.0,2022-04-28 07:30:00,0,1020.366971,1020.658514,1893.081494,2036.748797
3,PLT 4/28/22 B,4/28/22 7:30,PLT,surface,20.9,2078.410,22.0,1931.62999,30.0,2022-04-28 07:30:00,0,1020.366971,1020.658514,1893.073810,2036.342196
4,PLT 5/22/22 bottom,5/22/22 7:30,PLT,bottom,20.9,2138.395,22.0,1978.53868,30.0,2022-05-22 07:30:00,0,1020.366971,1020.658514,1939.046182,2095.113077


# Retrieve NB Fixed Station Monitoring Data
Remember, pH is in NBS

In [29]:


spreadsheet_key = '1XXFdxrZDcHTYUnLbQEZSgqvJpeNLIsKkE_HYJu1yTIo'
wks_name = 'MV'

#Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key(spreadsheet_key)
#Selecting which sheet to pulling the data
sheet = workbook.worksheet(wks_name)

#Pulling the data and transform it to the data frame
values = sheet.get_all_values()
ds = pd.DataFrame(values[1:], columns = values[0])
ds

,Site,Agency,Date,Time,C,Temp,SpC,Salinity,DO%,DO Conc,...,,,,,,,,,,
0,B6-S,gsovs,05/01/22,0:00,5/1/22 0:00,,,,,,...,,,,,,,,,,
1,B6-S,gsovs,05/01/22,0:15,5/1/22 0:15,,,,,,...,,,,,,,,,,
2,B6-S,gsovs,05/01/22,0:30,5/1/22 0:30,,,,,,...,,,,,,,,,,
3,B6-S,gsovs,05/01/22,0:45,5/1/22 0:45,,,,,,...,,,,,,,,,,
4,B6-S,gsovs,05/01/22,1:00,5/1/22 1:00,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19771,B6-S,gsovs,11/22/22,22:45,11/22/22 22:45,,,,,,...,,,,,,,,,,
19772,B6-S,gsovs,11/22/22,23:00,11/22/22 23:00,,,,,,...,,,,,,,,,,
19773,B6-S,gsovs,11/22/22,23:15,11/22/22 23:15,,,,,,...,,,,,,,,,,
19774,B6-S,gsovs,11/22/22,23:30,11/22/22 23:30,,,,,,...,,,,,,,,,,
